In [1]:
import ctypes
import numpy as np
import json

In [2]:
a = np.load("triangles_a.npy")
b = np.load("triangles_b.npy")

In [3]:
ifortlib = ctypes.CDLL("./src/fortran/ifort_clipping.dll")
def ifort_area_of_intersection(a, b):
    ntriangles, nvertex, ndim = a.shape
    out = np.empty(ntriangles, dtype=np.float64)
    function = ifortlib.area_of_intersection
    function(
        ctypes.pointer(ctypes.c_longlong(ndim)),
        ctypes.pointer(ctypes.c_longlong(nvertex)),
        ctypes.pointer(ctypes.c_longlong(ntriangles)),
        np.ctypeslib.as_ctypes(a),
        np.ctypeslib.as_ctypes(b),
        np.ctypeslib.as_ctypes(out),
    )
    return out

gfortranlib = ctypes.CDLL("./src/fortran/gfortran_clipping.dll")
def gfortran_area_of_intersection(a, b):
    ntriangles, nvertex, ndim = a.shape
    out = np.empty(ntriangles, dtype=np.float64)
    function = gfortranlib.area_of_intersection
    function(
        ctypes.pointer(ctypes.c_longlong(ndim)),
        ctypes.pointer(ctypes.c_longlong(nvertex)),
        ctypes.pointer(ctypes.c_longlong(ntriangles)),
        np.ctypeslib.as_ctypes(a),
        np.ctypeslib.as_ctypes(b),
        np.ctypeslib.as_ctypes(out),
    )
    return out

In [4]:
area_of_intersection = ifort_area_of_intersection

times = {}
for s in [1, 10, 100, 1_000, 10_000, 100_000, 1_000_000, 10_000_000]:
    res = %timeit -o area_of_intersection(a[:s], b[:s])
    times[s] = res.average
    
json.dump(times, open("timing/python-ifort.json", "w"))
np.save("answers/python-ifort.npy", area_of_intersection(a[:10_000], b[:10_000]))

29.1 µs ± 2.43 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
39.6 µs ± 3.72 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
47 µs ± 4.29 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
136 µs ± 19 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
1.02 ms ± 7.23 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
11.6 ms ± 1.01 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
101 ms ± 564 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
1.19 s ± 175 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
area_of_intersection = gfortran_area_of_intersection

times = {}
for s in [1, 10, 100, 1_000, 10_000, 100_000, 1_000_000, 10_000_000]:
    res = %timeit -o area_of_intersection(a[:s], b[:s])
    times[s] = res.average
    
json.dump(times, open("timing/python-gfortran.json", "w"))
np.save("answers/python-gfortran.npy", area_of_intersection(a[:10_000], b[:10_000]))

47.5 µs ± 5.7 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
48.7 µs ± 2.59 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
81.7 µs ± 10.1 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
425 µs ± 19.1 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
3.54 ms ± 361 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
35.1 ms ± 2.14 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
364 ms ± 16.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
2.88 s ± 211 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
